In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

/data/akmorrow/anaconda2/envs/python36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
### USER CONFIGURES ###
data_dir = "/data/akmorrow/tfti/t2t_data"
train_dir = "/data/akmorrow/tfti/t2t_train"
validation_file = "/data/akmorrow/tfti/data/deepseavalidation.txt"

In [96]:
sys.path.append("../tfti")
import tfti

# Reset graph for conistency.
tf.reset_default_graph()

# Prepare model.
problem_name = "genomics_binding_deepsea_gm12878"
model_name = "tfti_transformer"
hparams_set = "tfti_transformer_debug"
hparams_overrides_str = ""

data_dir = os.path.expanduser(data_dir)
#output_dir = os.path.expanduser(train_dir + f"/{problem_name}/{model_name}-{hparams_set}")
output_dir = os.path.expanduser("/data/akmorrow/tfti/t2t_train/genomics_binding_deepsea_gm12878/tfti_transformer-params_tfti_transformer_debug")

# Prepare model.
hparams = trainer_lib.create_hparams(hparams_set, hparams_overrides_str, data_dir, problem_name)
problem = registry.problem(problem_name)
encoders = problem.get_feature_encoders(data_dir)

# Prepare the model and the graph when model runs on features.
model = registry.model(model_name)(hparams, tf.estimator.ModeKeys.EVAL)

# Prepare features for feeding into the model.
inputs_ph = tf.placeholder(dtype=tf.string, shape=[ ])
targets_ph = tf.placeholder(dtype=tf.int64, shape=[problem.num_binary_predictions])
features = {"inputs": inputs_ph, "targets": targets_ph}
features = problem.preprocess_dev_example(features, tf.estimator.ModeKeys.EVAL, hparams)

# Introduce a dummy batch dimension.
for key in features.keys():
    features[key] = tf.expand_dims(features[key], 0)

logits, losses = model(features)
predictions = tf.nn.sigmoid(logits)
labels = features["targets"]

# Evaluation metrics we want to use.
set_auroc, _ = tfti.set_auroc(logits, labels, features)
set_auprc, _ = tfti.set_auprc(logits, labels, features)
average_auroc, _ = tfti.average_auroc(logits, labels, features)
average_auprc, _ = tfti.average_auprc(logits, labels, features)

# saver = tf.train.Saver()
sess = tf.InteractiveSession()

# Initialize AUC running average stuff.
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

# Load weights from checkpoint.
ckpts = tf.train.get_checkpoint_state(output_dir)
ckpt = ckpts.model_checkpoint_path
# saver.restore(sess, ckpt)

INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.


[2018-04-23 13:28:48,678] Unsetting shared_embedding_and_softmax_weights.


INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-04-23 13:28:48,679] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

[2018-04-23 13:28:48,891] Transforming 'targets' with binary_class_label_modality_8.targets_bottom


INFO:tensorflow:Building model body


[2018-04-23 13:28:48,909] Building model body


INFO:tensorflow:Transforming body output with binary_class_label_modality_8.top


[2018-04-23 13:28:50,091] Transforming body output with binary_class_label_modality_8.top


In [44]:
# load in test data
import pandas as pd
validation_data=pd.read_csv(validation_file, sep='\t',header=None)


In [68]:
dummy_inputs = "".join(np.random.choice(list("NACTG"), problem.input_sequence_length))
dummy_targets = np.random.choice(2, [problem.num_binary_predictions])

print(type(dummy_targets[0]))
print(dummy_targets.shape)

# Run on each line.
average_auroc_numpy = sess.run(average_auroc, feed_dict={
    inputs_ph: dummy_inputs,
    targets_ph: dummy_targets
})

# # Run on each line.
# set_auroc_numpy = sess.run(set_auroc, feed_dict={
#     inputs_ph: inputs,
#     targets_ph: targets
# })

# print(f"AUROC is: {average_auroc_numpy}")

# print(f"AUROC is: {set_auroc_numpy}")

<class 'numpy.int64'>
(919,)


In [ ]:
predictions_and_labels = []

for i in range(len(validation_data)):
    inputs = validation_data[0][i]
    targets = np.array(list(map(int, validation_data[1][i].split(','))))
    
    fetch = (predictions, labels)
    fetch_numpy = sess.run(fetch, feed_dict={
        inputs_ph: inputs,
        targets_ph: targets
    })
    
    predictions_and_labels.append(fetch_numpy)
    
predictions_and_labels = [(x.squeeze(), y.squeeze())
                          for (x, y) in predictions_and_labels]

In [ ]:
predictions_numpy = np.array(predictions_and_labels)[:, 0, :]
labels_numpy = np.array(predictions_and_labels)[:, 1, :]

In [ ]:
tfs = sorted(['SP1', 'Pol2-4H8', 'USF2', 'NRSF', 'RFX5', 'c-Myc', 'RXRA', 'EZH2', 'TBP', 'CHD1', 'Egr-1', 'SIN3A', 'GABP', 'CEBPB', 'Nrf1', 'p300', 'CTCF', 'ATF3', 'ATF2', 'Pol2', 'BCL11A', 'BRCA1', 'TCF12', 'SIX5', 'JunD', 'Rad21', 'YY1', 'USF-1', 'Max', 'TAF1', 'CHD2', 'Mxi1', 'SRF', 'Znf143'])

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(labels_numpy[:,0], predictions_numpy[:,0], pos_label=2)
metrics.auc(fpr, tpr)


plt.figure()

for i in range(len(tfs)):
    tf = tfs[i]

    # Compute micro-average ROC curve and ROC area
    fpr, tpr, _ = roc_curve(labels_numpy[:,0], predictions_numpy[:,0])
    roc_auc = auc(fpr, tpr)

    print(roc_auc)

    lw = 2
    plt.plot(fpr[2], tpr[2], color='darkorange',
             lw=lw, label='%s (%0.2f)' % (tf, 2))
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    
plt.show()

In [137]:
fpr, tpr

(array([0.        , 0.33333333, 1.        ]), array([nan, nan, nan]))

In [140]:
for i in range(len(tfs)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [148]:
dotprod =lambda K, L:reduce(lambda z1, z2: z1+z2, map(lambda x: reduce(lambda x1, x2: x1*x2, x), zip(K, L)))

In [149]:
dotprod([1, 2, 3, 4], [5, 6, 7, 8])

NameError: name 'reduce' is not defined